## Import necessary modules and data

In [1]:
import numpy as np
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score
from keras_metrics import precision
from sklearn.model_selection import cross_validate
from livelossplot import PlotLossesKeras
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras_metrics import precision
import random



Using TensorFlow backend.


## Wczytujemy poprzednio utworzone zbiory

In [2]:
X = pd.read_csv('../data/cleaned_data.csv', index_col=0)
y = pd.read_csv('../data/labels.csv', header = None)

X_test = pd.read_csv('../data/X_test.csv', index_col=0)
X_train = pd.read_csv('../data/X_train.csv', index_col=0)
X_train_std = pd.read_csv('../data/X_train_std.csv', index_col=0)
X_test_std = pd.read_csv('../data/X_test_std.csv', index_col=0)
y_train = pd.read_csv('../data/y_train.csv', index_col=0)
y_test = pd.read_csv('../data/y_test.csv', index_col=0)

Zbuduję przykładową sieć, która składa się z warstwy wejściowej, jednej warstwy ukrytej i  warstwy wyjściowej.
Kompilujemy model przy użyciu 

loss_function='binary_crossentropy';

optimizer='Adam'.


Naszą główną metryką będzie **precyzja** oznaczająca procent osób prawidłowie określonych jako zagrożone odejściem.

## Pierwsze podejście do modelu

In [8]:
model = Sequential()


model.add(Dense(11, activation='relu', input_shape=(10,)))
 
model.add(Dense(121, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[precision(),'accuracy'])
                   
model.fit(X_train_std, y_train,epochs=3, batch_size=1, verbose=1, validation_data=(X_test,y_test))

model.evaluate(X_test_std,y_test)

Train on 6700 samples, validate on 3300 samples
Epoch 1/3
6700/6700 [==============================] - 7s 1ms/step - loss: 0.4297 - precision: 0.6413662237872171 - acc: 0.8185 - val_loss: 3.2822 - val_precision: 0.0000e+00 - val_acc: 0.7964
Epoch 2/3
6700/6700 [==============================] - 7s 1ms/step - loss: 0.3717 - precision: 0.7299465239665848 - acc: 0.8476 - val_loss: 3.2822 - val_precision: 0.0000e+00 - val_acc: 0.7964
Epoch 3/3
3300/3300 [==============================] - 0s 17us/step


[0.3576833701494968, 0.7412790695519539, 0.8466666666666667]

## Sprawdźmy skuteczność naszej predykcji raportem klasyfikacyjnym


In [9]:
y_pred = model.predict_classes(X_test_std)
report = classification_report(y_test,y_pred)
report1 = classification_report(y_test,y_pred,output_dict=True)
print(report)
print(report1['0']['precision'])


              precision    recall  f1-score   support

           0       0.86      0.97      0.91      2628
           1       0.74      0.38      0.50       672

   micro avg       0.85      0.85      0.85      3300
   macro avg       0.80      0.67      0.71      3300
weighted avg       0.83      0.85      0.83      3300

0.8589309878213802


### Sprawdźmy, jak prezentują się wyniki w zależności od liczby neuronów w warstwie ukrytej.

In [10]:
nodes = [10, 30, 50, 70, 90, 110]
wyniki = {}
for i in nodes:
    model = 0

    model = Sequential()

    model.add(Dense(11, activation='relu', input_shape=(10,)))

    model.add(Dense(i, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=[precision(), 'accuracy'])

    model.fit(X_train_std, y_train, epochs=5, batch_size=1, verbose=1, validation_data=(X_test, y_test))
    print("Zakończono trenowanie modelu z {0} węzłami.".format(i))
    model.evaluate(X_test_std, y_test)
    y_pred = model.predict_classes(X_test_std)
    report = classification_report(y_test, y_pred, output_dict=True)
    wyniki.update({i: [report['0']['precision'], report['1']['precision']]})

print(wyniki)


Train on 6700 samples, validate on 3300 samples
Epoch 1/5
6700/6700 [==============================] - 7s 1ms/step - loss: 0.4367 - precision: 0.6375321335122026 - acc: 0.8122 - val_loss: 3.2822 - val_precision: 0.0000e+00 - val_acc: 0.7964
Epoch 2/5
6700/6700 [==============================] - 7s 1ms/step - loss: 0.3706 - precision: 0.7182910546437529 - acc: 0.8451 - val_loss: 3.2822 - val_precision: 0.0000e+00 - val_acc: 0.7964
Epoch 3/5
6700/6700 [==============================] - 7s 1ms/step - loss: 0.3585 - precision: 0.7308641974406341 - acc: 0.8521 - val_loss: 3.2933 - val_precision: 0.0000e+00 - val_acc: 0.7955
Epoch 4/5
6700/6700 [==============================] - 7s 1ms/step - loss: 0.3517 - precision: 0.7421686746093773 - acc: 0.8563 - val_loss: 3.2822 - val_precision: 0.0000e+00 - val_acc: 0.7964
Epoch 5/5
6700/6700 [==============================] - 7s 1ms/step - loss: 0.3488 - precision: 0.7724935731654893 - acc: 0.8596 - val_loss: 3.2822 - val_precision: 0.0000e+00 - val

## Wybieramy model z 90 węzłami - dał najlepsze wyniki

In [3]:
model = Sequential(name='model_1')

model.add(Dense(10, activation='relu', input_shape=(10,)))
model.add(Dropout(0.1, ))

model.add(Dense(90, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[precision(), 'accuracy'])

early_stopping_monitor = EarlyStopping(patience=2)
model.fit(X_train_std, y_train, epochs=30, batch_size=1, verbose=1, validation_data=(X_test, y_test),
          callbacks=[early_stopping_monitor])

model.evaluate(X_test_std, y_test)
y_pred = model.predict_classes(X_test_std)
report = classification_report(y_test, y_pred)
report1 = classification_report(y_test, y_pred, output_dict=True)
print(report)



tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> tp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> fp
Train on 6700 samples, validate on 3300 samples
Epoch 1/30
6700/6700 [==============================] - 8s 1ms/step - loss: 0.4401 - precision: 0.5113 - accuracy: 0.8161 - val_loss: 4952.8738 - val_precision: 0.0000e+00 - val_accuracy: 0.7964
Epoch 2/30
6700/6700 [==============================] - 8s 1ms/step - loss: 0.3944 - precision: 0.6957 - accuracy: 0.8379 - val_loss: 8124.6009 - val_precision: 0.0000e+00 - val_accuracy: 0.7964
Epoch 3/30
6700/6700 [==============================] - 8s 1ms/step - loss: 0.3835 - precision: 0.7191 - accuracy: 0.8461 - val_loss: 4179.3550 - val_precision: 0.2710 - val_accuracy: 0.7048
Epoch 4/30
6700/6700 [==============================] - 8s 1ms/step - loss: 0.3729 - precision: 0.7816 - accuracy: 0.8466 - val_loss: 4503.9204 - val_precision: 0.2476 - val_accuracy: 0.5845
Epoch 5/30
3300/3300 [====================

## Zapisujemy wytrenowany model

In [4]:
model.save("../saved_models/model_1.h5")

### autor: Tomasz Sołtysiak